In [9]:


import napari
from qtpy.QtWidgets import QSlider, QVBoxLayout, QWidget, QLabel
from qtpy.QtCore import Qt
from skimage.feature import peak_local_max
import numpy as np
from scipy.ndimage.measurements import center_of_mass, label

import sys
import os
cur=os.getcwd()
paths = [cur := os.path.dirname(cur) for _ in range(3)]
sys.path.insert(0, paths[0])
from Helpers import loadData

C:\Users\Lab User\AppData\Local\Temp\ipykernel_18032\4127699419.py:6: DeprecationWarning: Please import `center_of_mass` from the `scipy.ndimage` namespace; the `scipy.ndimage.measurements` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.ndimage.measurements import center_of_mass, label
C:\Users\Lab User\AppData\Local\Temp\ipykernel_18032\4127699419.py:6: DeprecationWarning: Please import `label` from the `scipy.ndimage` namespace; the `scipy.ndimage.measurements` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.ndimage.measurements import center_of_mass, label


In [17]:
data = loadData(os.path.join(paths[0],'Data'),'convMap_17.hdf5')
peaks = peak_local_max(data, min_distance=7)

In [ ]:
volume = np.copy(data)
adjust = np.copy(peaks)

In [19]:
viewer = napari.Viewer(ndisplay=3)
image_layer = viewer.add_image(volume, name='Volume', colormap='gray') # Plots in [z,y,x]
points_layer = viewer.add_points(adjust, size=7, face_color='red', name='Centers')

dock_widget = QWidget()
layout = QVBoxLayout()

label_min = QLabel("Min Z")
slider_min = QSlider(Qt.Horizontal)
slider_min.setMinimum(0)
slider_min.setMaximum(volume.shape[0]-1) # Uses z coord of data -1 to find max
slider_min.setValue(0)

label_max = QLabel("Max Z")
slider_max = QSlider(Qt.Horizontal)
slider_max.setMinimum(0)
slider_max.setMaximum(volume.shape[0]-1)
slider_max.setValue(volume.shape[0]-1)

layout.addWidget(label_min)
layout.addWidget(slider_min)
layout.addWidget(label_max)
layout.addWidget(slider_max)
dock_widget.setLayout(layout)
viewer.window.add_dock_widget(dock_widget, area='right')

def update_view():
    
    zmin = slider_min.value()
    zmax = slider_max.value()

    # Clip points to slab
    filtered_points = adjust[(adjust[:, 0] >= zmin) & (adjust[:, 0] <= zmax)]
    points_layer.data = filtered_points  # z, y, x order

    # Clip volume to slab
    filtered_data = volume[zmin:zmax+1, :, :]
    image_layer.data = np.pad(filtered_data, pad_width=[[zmin, volume.shape[0] - zmax - 1], [0, 0], [0, 0]], mode='constant', constant_values=0)

slider_min.valueChanged.connect(update_view)
slider_max.valueChanged.connect(update_view)
update_view()  # initialize

napari.run()